### 209

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')

100%|██████████| 3/3 [00:00<00:00, 39.15it/s]


In [3]:
out_id_list = df_train[df_train[target]<-30][key].values
len(out_id_list)

2207

In [6]:
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
# df_hist = utils.read_df_pkl('../input/hist_exist*')
df_hist = reduce_mem_usage(df_hist)
# df_new = utils.read_df_pkl('../input/new_mer*0*')
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
# df_new = utils.read_df_pkl('../input/new_exist*')
df_new = reduce_mem_usage(df_new)
df_trans = pd.concat([df_hist, df_new], axis=0)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
# df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
df_trans = df_trans.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])

df_trans = df_trans[df_trans[key].isin(out_id_list)]
del df_hist, df_new
gc.collect()

1978

In [7]:
m_id = 'merchant_id'
mc_id = 'merchant_category_id'
lag_col = 'org_month_lag'
lag_col = 'month_lag'
# past_1 = 0
# past_2 = 3
# future_1 = 4
# future_2 = 5

past_2_list = np.array([-1,-2,-3,-4,-5,-6])
past_1_list = np.zeros(len(past_2_list)) -13

future_1_list = past_2_list.copy() + 1
future_2_list = future_1_list.copy() + 1

feat_list = []
fname_list = [ f'auth1_new' ,f'auth0_new']
    

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

    
def feat_agg(df, fname):
# for df, fname in zip(df_list, fname_list):
    global train_test
    
    aggs = {}
    if fname.count('cat1'):
        df = pd.get_dummies(df, columns=['category_3'])
        aggs = {
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    else:
        df = pd.get_dummies(df, columns=['category_2', 'category_3'])
        aggs = {
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    
    if not(fname.count('cat')):
        aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
#     aggs['month_lag'] = ['mean', 'std']
    aggs['org_month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    monthly=True
#     monthly=False
    
#     Monthly Agg
    if monthly:
        new_columns = get_new_columns(fname + '_monthly_avg', aggs)
        month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
            
        month_agg.columns = new_columns
        month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
        month_agg.reset_index(drop=False,inplace=True)
        
        train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
        
        del df_agg, month_agg
        gc.collect()
    else:
        train_test = train_test.join(df_agg.set_index(key))
        del df_agg
        gc.collect()
    
    train_test = impute_agg(train_test)
    train_test.columns =  [col.replace('.', '_') for col in train_test.columns]
    
    display(train_test.head(3))
    
    feat_list.append(train_test.copy())
    del train_test
    gc.collect()
    
#     print(f"Saving features... {fname} {train_test.shape}")
#     p_list = Parallel(n_jobs=-1)([delayed(save_feature)(col) for col in train_test.columns])

#     for col in tqdm(train_test.columns):
#         ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
#         if col in ignore_features: return
#         if (col.count('feature_')):return
#         if (col.count('purchase_date')):return
#         feature = train_test[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)


def save_feature(col):
    ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
    if col in ignore_features: return
    if (col.count('feature_')):return
    if (col.count('purchase_date')):return
    feature = train_test[col].fillna(-1).astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    

def impute_agg(df):
    for col in df.columns:
        if col.count('category_1'):
            df[col].fillna(-1, inplace=True)
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
    
    
# for df, fname in zip([auth1, auth0], fname_list):
for df, fname_base in zip([auth1], fname_list):
    
    for past_1, past_2 in zip(past_1_list, past_2_list):
        
        future_1 = past_2+1
        future_2 = past_2+2
#         fname = f'180_l{future_1}{future_2}'
        fname = f'_lag{future_1}_{future_2}' + '_' + fname_base

        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        df_train.set_index(key, inplace=True)
        df_test.set_index(key, inplace=True)
        train_test = pd.concat([df_train, df_test], axis=0)
        train_test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        
        future = df[future_1<= df[lag_col]][df[lag_col]<=future_2][[key, mc_id, m_id]].drop_duplicates()
        past = df[past_1<= df[lag_col]][df[lag_col]<=past_2][[key, mc_id, m_id]].drop_duplicates()
        
        # inner joinで過去に存在したmerchantをdf_nowに残し、そのindexをdf_nowから除外することでnew_merchantのみ残す。天才！
        future['index'] = np.arange(len(future))
        mer_same = future.merge(past, how='inner', on=[key, m_id, mc_id])
        past_mer_index = mer_same['index'].values
        future = future.loc[~future['index'].isin(past_mer_index), :]
        
        future.drop('index', axis=1, inplace=True)
        future.dropna(inplace=True)
        
        # ここでtransactionをinner joinしてnew_merchantに対する購買履歴のみのテーブルができる
        new_hist = future.merge(df, how='inner', on=[key, mc_id, m_id])
        
        feat_agg(new_hist, fname)

        
# for df in feat_list:
#     try:
#         df.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
#     except KeyError:
#         pass
    
#     try:
#         df.set_index(key, inplace=True)
#     except KeyError:
#         pass
    
#     train_test = train_test.join(df)

# train_test = impute_agg(train_test)
# display(train_test.head())


100%|██████████| 3/3 [00:00<00:00, 168.12it/s]

,lag0_1_auth1_new_category_2_1_0_mean,lag0_1_auth1_new_category_2_2_0_mean,lag0_1_auth1_new_category_2_3_0_mean,lag0_1_auth1_new_category_2_4_0_mean,lag0_1_auth1_new_category_2_5_0_mean,lag0_1_auth1_new_category_3_A_mean,lag0_1_auth1_new_category_3_B_mean,lag0_1_auth1_new_category_3_C_mean,lag0_1_auth1_new_category_1_sum,lag0_1_auth1_new_category_1_mean,lag0_1_auth1_new_org_month_lag_mean,lag0_1_auth1_new_org_month_lag_std,lag0_1_auth1_new_yyyy_week_nunique,lag0_1_auth1_new_purchase_date_max,lag0_1_auth1_new_purchase_date_min,lag0_1_auth1_new_month_diff_mean,lag0_1_auth1_new_month_diff_std,lag0_1_auth1_new_purchase_amount_sum,lag0_1_auth1_new_purchase_amount_max,lag0_1_auth1_new_purchase_amount_min,lag0_1_auth1_new_purchase_amount_mean,lag0_1_auth1_new_installments_sum,lag0_1_auth1_new_installments_max,lag0_1_auth1_new_installments_min,lag0_1_auth1_new_installments_mean,lag0_1_auth1_new_installments_std,lag0_1_auth1_new_merchant_id_nunique,lag0_1_auth1_new_merchant_category_id_nunique,lag0_1_auth1_new_card_id_size,lag0_1_auth1_new_city_id_nunique,lag0_1_auth1_new_state_id_nunique,lag0_1_auth1_new_subsector_id_nunique,lag0_1_auth1_new_this_term,lag0_1_auth1_new_freq_per_this_term,lag0_1_auth1_new_amount_per_this_term,lag0_1_auth1_new_instthisments_per_this_term,lag0_1_auth1_new_amount_per_installments_sum,lag0_1_auth1_new_amount_per_installments_mean,lag0_1_auth1_new_monthly_avg_category_2_1_0_mean,lag0_1_auth1_new_monthly_avg_category_2_2_0_mean,lag0_1_auth1_new_monthly_avg_category_2_3_0_mean,lag0_1_auth1_new_monthly_avg_category_2_4_0_mean,lag0_1_auth1_new_monthly_avg_category_2_5_0_mean,lag0_1_auth1_new_monthly_avg_category_3_A_mean,lag0_1_auth1_new_monthly_avg_category_3_B_mean,lag0_1_auth1_new_monthly_avg_category_3_C_mean,lag0_1_auth1_new_monthly_avg_category_1_sum,lag0_1_auth1_new_monthly_avg_category_1_mean,lag0_1_auth1_new_monthly_avg_org_month_lag_mean,lag0_1_auth1_new_monthly_avg_org_month_lag_std,lag0_1_auth1_new_monthly_avg_yyyy_week_nunique,lag0_1_auth1_new_monthly_avg_month_diff_mean,lag0_1_auth1_new_monthly_avg_month_diff_std,lag0_1_auth1_new_monthly_avg_purchase_amount_sum,lag0_1_auth1_new_monthly_avg_purchase_amount_max,lag0_1_auth1_new_monthly_avg_purchase_amount_min,lag0_1_auth1_new_monthly_avg_purchase_amount_mean,lag0_1_auth1_new_monthly_avg_installments_sum,lag0_1_auth1_new_monthly_avg_installments_max,lag0_1_auth1_new_monthly_avg_installments_min,lag0_1_auth1_new_monthly_avg_installments_mean,lag0_1_auth1_new_monthly_avg_installments_std,lag0_1_auth1_new_monthly_avg_merchant_id_nunique,lag0_1_auth1_new_monthly_avg_merchant_category_id_nunique,lag0_1_auth1_new_monthly_avg_card_id_size,lag0_1_auth1_new_monthly_avg_city_id_nunique,lag0_1_auth1_new_monthly_avg_state_id_nunique,lag0_1_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,8.750000,0.447214,6.0,2018-03-31 15:08:29,2018-02-02 08:47:57,52.437500,19.626407,2.390625,0.450928,0.007973,0.149414,0.0,0.0,0.0,0.000000,0.000000,16.0,12.0,16.0,3.0,1.0,9.0,57.0,0.275862,0.041218,0.000000,2.390625,0.149414,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,8.5,0.0,3.0,62.208333,7.542581,1.196289,0.255615,0.015228,0.109375,0.0,0.0,0.0,0.000000,0.000000,8.0,6.0,8.0,2.0,1.0,5.0
C_ID_3d0044924f,1.0,0.0,0.0,0.0,0.0,0.0,0.769231,0.153846,0.0,0.0,12.230769,0.438529,6.0,2018-02-05 17:17:01,2018-01-05 19:17:52,96.230769,9.283981,0.905273,0.314697,0.007820,0.069641,14.0,2.0,0.0,1.077148,0.493408,13.0,10.0,13.0,3.0,1.0,10.0,30.0,0.419355,0.029202,0.451613,0.060364,0.033508,1.0,0.0,0.0,0.0,0.0,0.0,0.85,0.1,0.0,0.0,12.5,0.0,3.0,92.416667,5.100514,0.452637,0.164917,0.009171,0.049530,7.0,1.5,0.5,1.049805,0.283936,6.5,5.5,6.5,2.0,1.0,5.5
C_ID_d639edf6cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-15.000000,-15.000000,-1.0,NaT,NaT,553.000000,143.133514,-1.000000,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


100%|██████████| 3/3 [00:00<00:00, 169.18it/s]

,lag-1_0_auth1_new_category_2_1_0_mean,lag-1_0_auth1_new_category_2_2_0_mean,lag-1_0_auth1_new_category_2_3_0_mean,lag-1_0_auth1_new_category_2_4_0_mean,lag-1_0_auth1_new_category_2_5_0_mean,lag-1_0_auth1_new_category_3_A_mean,lag-1_0_auth1_new_category_3_B_mean,lag-1_0_auth1_new_category_3_C_mean,lag-1_0_auth1_new_category_1_sum,lag-1_0_auth1_new_category_1_mean,lag-1_0_auth1_new_org_month_lag_mean,lag-1_0_auth1_new_org_month_lag_std,lag-1_0_auth1_new_yyyy_week_nunique,lag-1_0_auth1_new_purchase_date_max,lag-1_0_auth1_new_purchase_date_min,lag-1_0_auth1_new_month_diff_mean,lag-1_0_auth1_new_month_diff_std,lag-1_0_auth1_new_purchase_amount_sum,lag-1_0_auth1_new_purchase_amount_max,lag-1_0_auth1_new_purchase_amount_min,lag-1_0_auth1_new_purchase_amount_mean,lag-1_0_auth1_new_installments_sum,lag-1_0_auth1_new_installments_max,lag-1_0_auth1_new_installments_min,lag-1_0_auth1_new_installments_mean,lag-1_0_auth1_new_installments_std,lag-1_0_auth1_new_merchant_id_nunique,lag-1_0_auth1_new_merchant_category_id_nunique,lag-1_0_auth1_new_card_id_size,lag-1_0_auth1_new_city_id_nunique,lag-1_0_auth1_new_state_id_nunique,lag-1_0_auth1_new_subsector_id_nunique,lag-1_0_auth1_new_this_term,lag-1_0_auth1_new_freq_per_this_term,lag-1_0_auth1_new_amount_per_this_term,lag-1_0_auth1_new_instthisments_per_this_term,lag-1_0_auth1_new_amount_per_installments_sum,lag-1_0_auth1_new_amount_per_installments_mean,lag-1_0_auth1_new_monthly_avg_category_2_1_0_mean,lag-1_0_auth1_new_monthly_avg_category_2_2_0_mean,lag-1_0_auth1_new_monthly_avg_category_2_3_0_mean,lag-1_0_auth1_new_monthly_avg_category_2_4_0_mean,lag-1_0_auth1_new_monthly_avg_category_2_5_0_mean,lag-1_0_auth1_new_monthly_avg_category_3_A_mean,lag-1_0_auth1_new_monthly_avg_category_3_B_mean,lag-1_0_auth1_new_monthly_avg_category_3_C_mean,lag-1_0_auth1_new_monthly_avg_category_1_sum,lag-1_0_auth1_new_monthly_avg_category_1_mean,lag-1_0_auth1_new_monthly_avg_org_month_lag_mean,lag-1_0_auth1_new_monthly_avg_org_month_lag_std,lag-1_0_auth1_new_monthly_avg_yyyy_week_nunique,lag-1_0_auth1_new_monthly_avg_month_diff_mean,lag-1_0_auth1_new_monthly_avg_month_diff_std,lag-1_0_auth1_new_monthly_avg_purchase_amount_sum,lag-1_0_auth1_new_monthly_avg_purchase_amount_max,lag-1_0_auth1_new_monthly_avg_purchase_amount_min,lag-1_0_auth1_new_monthly_avg_purchase_amount_mean,lag-1_0_auth1_new_monthly_avg_installments_sum,lag-1_0_auth1_new_monthly_avg_installments_max,lag-1_0_auth1_new_monthly_avg_installments_min,lag-1_0_auth1_new_monthly_avg_installments_mean,lag-1_0_auth1_new_monthly_avg_installments_std,lag-1_0_auth1_new_monthly_avg_merchant_id_nunique,lag-1_0_auth1_new_monthly_avg_merchant_category_id_nunique,lag-1_0_auth1_new_monthly_avg_card_id_size,lag-1_0_auth1_new_monthly_avg_city_id_nunique,lag-1_0_auth1_new_monthly_avg_state_id_nunique,lag-1_0_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,0.727273,0.0,0.0,0.0,0.272727,1.0,0.000000,0.000000,0.0,0.000000,7.363636,0.504525,6.0,2018-02-12 15:46:30,2018-01-03 13:47:14,98.454545,15.532956,1.099609,0.515625,0.007973,0.099915,0.0,0.0,0.0,0.000000,0.000000,11.0,11.0,11.0,4.0,3.0,9.0,40.0,0.268293,0.026820,0.000000,1.099609,0.099915,0.785714,0.0,0.0,0.0,0.214286,1.0,0.000000,0.000000,0.0,0.000000,7.5,0.0,3.5,94.875000,7.428713,0.549805,0.287842,0.011505,0.084778,0.0,0.0,0.0,0.000000,0.000000,5.5,5.5,5.5,2.5,2.0,5.0
C_ID_3d0044924f,0.925926,0.0,0.0,0.0,0.000000,0.0,0.740741,0.185185,2.0,0.074074,11.370370,0.492103,8.0,2018-01-28 10:38:06,2017-12-06 14:24:30,118.962963,17.236994,2.000000,0.314697,0.004517,0.074036,31.0,3.0,0.0,1.148438,0.601562,25.0,15.0,27.0,5.0,2.0,12.0,52.0,0.509434,0.037736,0.584906,0.062500,0.034454,0.941176,0.0,0.0,0.0,0.000000,0.0,0.732353,0.188235,1.0,0.058824,11.5,0.0,4.0,114.955882,8.181299,1.000000,0.275391,0.007523,0.076660,15.5,2.5,0.0,1.138672,0.601562,12.5,9.0,13.5,3.5,1.5,7.5
C_ID_d639edf6cd,0.000000,0.0,0.0,0.0,1.000000,1.0,0.000000,0.000000


100%|██████████| 3/3 [00:00<00:00, 111.99it/s]

...

,lag-3_-2_auth1_new_category_2_1_0_mean,lag-3_-2_auth1_new_category_2_2_0_mean,lag-3_-2_auth1_new_category_2_3_0_mean,lag-3_-2_auth1_new_category_2_4_0_mean,lag-3_-2_auth1_new_category_2_5_0_mean,lag-3_-2_auth1_new_category_3_A_mean,lag-3_-2_auth1_new_category_3_B_mean,lag-3_-2_auth1_new_category_3_C_mean,lag-3_-2_auth1_new_category_1_sum,lag-3_-2_auth1_new_category_1_mean,lag-3_-2_auth1_new_org_month_lag_mean,lag-3_-2_auth1_new_org_month_lag_std,lag-3_-2_auth1_new_yyyy_week_nunique,lag-3_-2_auth1_new_purchase_date_max,lag-3_-2_auth1_new_purchase_date_min,lag-3_-2_auth1_new_month_diff_mean,lag-3_-2_auth1_new_month_diff_std,lag-3_-2_auth1_new_purchase_amount_sum,lag-3_-2_auth1_new_purchase_amount_max,lag-3_-2_auth1_new_purchase_amount_min,lag-3_-2_auth1_new_purchase_amount_mean,lag-3_-2_auth1_new_installments_sum,lag-3_-2_auth1_new_installments_max,lag-3_-2_auth1_new_installments_min,lag-3_-2_auth1_new_installments_mean,lag-3_-2_auth1_new_installments_std,lag-3_-2_auth1_new_merchant_id_nunique,lag-3_-2_auth1_new_merchant_category_id_nunique,lag-3_-2_auth1_new_card_id_size,lag-3_-2_auth1_new_city_id_nunique,lag-3_-2_auth1_new_state_id_nunique,lag-3_-2_auth1_new_subsector_id_nunique,lag-3_-2_auth1_new_this_term,lag-3_-2_auth1_new_freq_per_this_term,lag-3_-2_auth1_new_amount_per_this_term,lag-3_-2_auth1_new_instthisments_per_this_term,lag-3_-2_auth1_new_amount_per_installments_sum,lag-3_-2_auth1_new_amount_per_installments_mean,lag-3_-2_auth1_new_monthly_avg_category_2_1_0_mean,lag-3_-2_auth1_new_monthly_avg_category_2_2_0_mean,lag-3_-2_auth1_new_monthly_avg_category_2_3_0_mean,lag-3_-2_auth1_new_monthly_avg_category_2_4_0_mean,lag-3_-2_auth1_new_monthly_avg_category_2_5_0_mean,lag-3_-2_auth1_new_monthly_avg_category_3_A_mean,lag-3_-2_auth1_new_monthly_avg_category_3_B_mean,lag-3_-2_auth1_new_monthly_avg_category_3_C_mean,lag-3_-2_auth1_new_monthly_avg_category_1_sum,lag-3_-2_auth1_new_monthly_avg_category_1_mean,lag-3_-2_auth1_new_monthly_avg_org_month_lag_mean,lag-3_-2_auth1_new_monthly_avg_org_month_lag_std,lag-3_-2_auth1_new_monthly_avg_yyyy_week_nunique,lag-3_-2_auth1_new_monthly_avg_month_diff_mean,lag-3_-2_auth1_new_monthly_avg_month_diff_std,lag-3_-2_auth1_new_monthly_avg_purchase_amount_sum,lag-3_-2_auth1_new_monthly_avg_purchase_amount_max,lag-3_-2_auth1_new_monthly_avg_purchase_amount_min,lag-3_-2_auth1_new_monthly_avg_purchase_amount_mean,lag-3_-2_auth1_new_monthly_avg_installments_sum,lag-3_-2_auth1_new_monthly_avg_installments_max,lag-3_-2_auth1_new_monthly_avg_installments_min,lag-3_-2_auth1_new_monthly_avg_installments_mean,lag-3_-2_auth1_new_monthly_avg_installments_std,lag-3_-2_auth1_new_monthly_avg_merchant_id_nunique,lag-3_-2_auth1_new_monthly_avg_merchant_category_id_nunique,lag-3_-2_auth1_new_monthly_avg_card_id_size,lag-3_-2_auth1_new_monthly_avg_city_id_nunique,lag-3_-2_auth1_new_monthly_avg_state_id_nunique,lag-3_-2_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,6.069767,0.768280,12.0,2018-02-17 09:39:17,2017-11-11 00:00:00,131.279070,22.983768,9.406250,2.000000,0.012032,0.218872,0.0,0.0,0.0,0.000000,0.000000,31.0,20.0,43.0,1.0,1.0,14.0,98.0,0.434343,0.095013,0.000000,9.406250,0.218872,1.000000,0.0,0.0,0.0,0.0,1.0,0.00000,0.00000,0.00,0.000000,6.5,0.0,3.25,118.096429,7.609796,2.353516,0.670410,0.049072,0.160156,0.00,0.00,0.00,0.0000,0.000000,9.25,7.25,10.75,1.00,1.0,5.50
C_ID_3d0044924f,0.954545,0.0,0.0,0.0,0.0,0.0,0.863636,0.136364,1.0,0.045455,9.409091,0.796366,9.0,2018-01-23 18:49:01,2017-10-15 17:20:41,179.136364,25.392716,2.396484,0.330322,0.006020,0.108887,29.0,4.0,1.0,1.318359,0.838867,19.0,14.0,22.0,4.0,3.0,9.0,100.0,0.217822,0.023728,0.287129,0.079895,0.046967,0.984375,0.0,0.0,0.0,0.0,0.0,0.71875,0.28125,0.25,0.015625,10.5,0.0,2.25,144.546875,6.629421,0.599121,0.219604,0.127319,0.156494,7.25,2.25,1.75,1.8125,0.341553,5.25,4.25,5.50,1.75,1.5,3.25
C_ID_d


100%|██████████| 3/3 [00:00<00:00, 171.26it/s]

,lag-4_-3_auth1_new_category_2_1_0_mean,lag-4_-3_auth1_new_category_2_2_0_mean,lag-4_-3_auth1_new_category_2_3_0_mean,lag-4_-3_auth1_new_category_2_4_0_mean,lag-4_-3_auth1_new_category_2_5_0_mean,lag-4_-3_auth1_new_category_3_A_mean,lag-4_-3_auth1_new_category_3_B_mean,lag-4_-3_auth1_new_category_3_C_mean,lag-4_-3_auth1_new_category_1_sum,lag-4_-3_auth1_new_category_1_mean,lag-4_-3_auth1_new_org_month_lag_mean,lag-4_-3_auth1_new_org_month_lag_std,lag-4_-3_auth1_new_yyyy_week_nunique,lag-4_-3_auth1_new_purchase_date_max,lag-4_-3_auth1_new_purchase_date_min,lag-4_-3_auth1_new_month_diff_mean,lag-4_-3_auth1_new_month_diff_std,lag-4_-3_auth1_new_purchase_amount_sum,lag-4_-3_auth1_new_purchase_amount_max,lag-4_-3_auth1_new_purchase_amount_min,lag-4_-3_auth1_new_purchase_amount_mean,lag-4_-3_auth1_new_installments_sum,lag-4_-3_auth1_new_installments_max,lag-4_-3_auth1_new_installments_min,lag-4_-3_auth1_new_installments_mean,lag-4_-3_auth1_new_installments_std,lag-4_-3_auth1_new_merchant_id_nunique,lag-4_-3_auth1_new_merchant_category_id_nunique,lag-4_-3_auth1_new_card_id_size,lag-4_-3_auth1_new_city_id_nunique,lag-4_-3_auth1_new_state_id_nunique,lag-4_-3_auth1_new_subsector_id_nunique,lag-4_-3_auth1_new_this_term,lag-4_-3_auth1_new_freq_per_this_term,lag-4_-3_auth1_new_amount_per_this_term,lag-4_-3_auth1_new_instthisments_per_this_term,lag-4_-3_auth1_new_amount_per_installments_sum,lag-4_-3_auth1_new_amount_per_installments_mean,lag-4_-3_auth1_new_monthly_avg_category_2_1_0_mean,lag-4_-3_auth1_new_monthly_avg_category_2_2_0_mean,lag-4_-3_auth1_new_monthly_avg_category_2_3_0_mean,lag-4_-3_auth1_new_monthly_avg_category_2_4_0_mean,lag-4_-3_auth1_new_monthly_avg_category_2_5_0_mean,lag-4_-3_auth1_new_monthly_avg_category_3_A_mean,lag-4_-3_auth1_new_monthly_avg_category_3_B_mean,lag-4_-3_auth1_new_monthly_avg_category_3_C_mean,lag-4_-3_auth1_new_monthly_avg_category_1_sum,lag-4_-3_auth1_new_monthly_avg_category_1_mean,lag-4_-3_auth1_new_monthly_avg_org_month_lag_mean,lag-4_-3_auth1_new_monthly_avg_org_month_lag_std,lag-4_-3_auth1_new_monthly_avg_yyyy_week_nunique,lag-4_-3_auth1_new_monthly_avg_month_diff_mean,lag-4_-3_auth1_new_monthly_avg_month_diff_std,lag-4_-3_auth1_new_monthly_avg_purchase_amount_sum,lag-4_-3_auth1_new_monthly_avg_purchase_amount_max,lag-4_-3_auth1_new_monthly_avg_purchase_amount_min,lag-4_-3_auth1_new_monthly_avg_purchase_amount_mean,lag-4_-3_auth1_new_monthly_avg_installments_sum,lag-4_-3_auth1_new_monthly_avg_installments_max,lag-4_-3_auth1_new_monthly_avg_installments_min,lag-4_-3_auth1_new_monthly_avg_installments_mean,lag-4_-3_auth1_new_monthly_avg_installments_std,lag-4_-3_auth1_new_monthly_avg_merchant_id_nunique,lag-4_-3_auth1_new_monthly_avg_merchant_category_id_nunique,lag-4_-3_auth1_new_monthly_avg_card_id_size,lag-4_-3_auth1_new_monthly_avg_city_id_nunique,lag-4_-3_auth1_new_monthly_avg_state_id_nunique,lag-4_-3_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,5.151515,1.394660,14.0,2018-02-21 18:49:39,2017-10-01 15:38:43,161.545455,42.961531,3.566406,0.300537,0.010529,0.108032,0.0,0.0,0.0,0.000000,0.000000,18.0,15.0,33.0,3.0,1.0,13.0,143.0,0.229167,0.024767,0.000000,3.566406,0.108032,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,5.75,0.0,3.75,144.076786,8.921070,0.891602,0.279785,0.021866,0.117493,0.00,0.00,0.00,0.000000,0.000000,6.50,5.75,8.25,1.50,1.00,5.50
C_ID_3d0044924f,0.866667,0.0,0.0,0.0,0.0,0.0,0.7,0.3,4.0,0.133333,8.866667,1.008014,10.0,2018-01-16 20:03:30,2017-09-04 16:50:16,197.700000,31.008508,5.296875,1.341797,0.006020,0.176514,61.0,10.0,1.0,2.033203,2.326172,26.0,20.0,30.0,6.0,3.0,13.0,134.0,0.222222,0.039236,0.451852,0.085449,0.058197,0.916193,0.0,0.0,0.0,0.0,0.0,0.480114,0.519886,1.0,0.083807,9.75,0.0,2.50,171.217330,6.479688,1.324219,0.552734,0.062469,0.189697,15.25,4.75,1.25,2.210938,2.103516,7.25,6.50,7.50,2.75,1.75,5.25
C_ID_d639edf6


100%|██████████| 3/3 [00:00<00:00, 175.33it/s]

,lag-5_-4_auth1_new_category_2_1_0_mean,lag-5_-4_auth1_new_category_2_2_0_mean,lag-5_-4_auth1_new_category_2_3_0_mean,lag-5_-4_auth1_new_category_2_4_0_mean,lag-5_-4_auth1_new_category_2_5_0_mean,lag-5_-4_auth1_new_category_3_A_mean,lag-5_-4_auth1_new_category_3_B_mean,lag-5_-4_auth1_new_category_3_C_mean,lag-5_-4_auth1_new_category_1_sum,lag-5_-4_auth1_new_category_1_mean,lag-5_-4_auth1_new_org_month_lag_mean,lag-5_-4_auth1_new_org_month_lag_std,lag-5_-4_auth1_new_yyyy_week_nunique,lag-5_-4_auth1_new_purchase_date_max,lag-5_-4_auth1_new_purchase_date_min,lag-5_-4_auth1_new_month_diff_mean,lag-5_-4_auth1_new_month_diff_std,lag-5_-4_auth1_new_purchase_amount_sum,lag-5_-4_auth1_new_purchase_amount_max,lag-5_-4_auth1_new_purchase_amount_min,lag-5_-4_auth1_new_purchase_amount_mean,lag-5_-4_auth1_new_installments_sum,lag-5_-4_auth1_new_installments_max,lag-5_-4_auth1_new_installments_min,lag-5_-4_auth1_new_installments_mean,lag-5_-4_auth1_new_installments_std,lag-5_-4_auth1_new_merchant_id_nunique,lag-5_-4_auth1_new_merchant_category_id_nunique,lag-5_-4_auth1_new_card_id_size,lag-5_-4_auth1_new_city_id_nunique,lag-5_-4_auth1_new_state_id_nunique,lag-5_-4_auth1_new_subsector_id_nunique,lag-5_-4_auth1_new_this_term,lag-5_-4_auth1_new_freq_per_this_term,lag-5_-4_auth1_new_amount_per_this_term,lag-5_-4_auth1_new_instthisments_per_this_term,lag-5_-4_auth1_new_amount_per_installments_sum,lag-5_-4_auth1_new_amount_per_installments_mean,lag-5_-4_auth1_new_monthly_avg_category_2_1_0_mean,lag-5_-4_auth1_new_monthly_avg_category_2_2_0_mean,lag-5_-4_auth1_new_monthly_avg_category_2_3_0_mean,lag-5_-4_auth1_new_monthly_avg_category_2_4_0_mean,lag-5_-4_auth1_new_monthly_avg_category_2_5_0_mean,lag-5_-4_auth1_new_monthly_avg_category_3_A_mean,lag-5_-4_auth1_new_monthly_avg_category_3_B_mean,lag-5_-4_auth1_new_monthly_avg_category_3_C_mean,lag-5_-4_auth1_new_monthly_avg_category_1_sum,lag-5_-4_auth1_new_monthly_avg_category_1_mean,lag-5_-4_auth1_new_monthly_avg_org_month_lag_mean,lag-5_-4_auth1_new_monthly_avg_org_month_lag_std,lag-5_-4_auth1_new_monthly_avg_yyyy_week_nunique,lag-5_-4_auth1_new_monthly_avg_month_diff_mean,lag-5_-4_auth1_new_monthly_avg_month_diff_std,lag-5_-4_auth1_new_monthly_avg_purchase_amount_sum,lag-5_-4_auth1_new_monthly_avg_purchase_amount_max,lag-5_-4_auth1_new_monthly_avg_purchase_amount_min,lag-5_-4_auth1_new_monthly_avg_purchase_amount_mean,lag-5_-4_auth1_new_monthly_avg_installments_sum,lag-5_-4_auth1_new_monthly_avg_installments_max,lag-5_-4_auth1_new_monthly_avg_installments_min,lag-5_-4_auth1_new_monthly_avg_installments_mean,lag-5_-4_auth1_new_monthly_avg_installments_std,lag-5_-4_auth1_new_monthly_avg_merchant_id_nunique,lag-5_-4_auth1_new_monthly_avg_merchant_category_id_nunique,lag-5_-4_auth1_new_monthly_avg_card_id_size,lag-5_-4_auth1_new_monthly_avg_city_id_nunique,lag-5_-4_auth1_new_monthly_avg_state_id_nunique,lag-5_-4_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1.000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000,4.975610,1.573019,18.0,2018-02-25 09:31:15,2017-09-02 12:48:59,167.292683,50.216653,5.031250,0.480225,0.009026,0.122742,0.0,0.0,0.0,0.000,0.000000,19.0,16.0,41.0,3.0,1.0,12.0,175.0,0.232955,0.028587,0.000000,5.031250,0.122742,1.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.00,0.000000,5.50,0.0,3.333333,150.985450,7.507460,0.838867,0.262939,0.023193,0.122803,0.00,0.0,0.0,0.000000,0.000000,5.00,4.50,6.833333,1.333333,1.00,4.0
C_ID_3d0044924f,0.875,0.0,0.0,0.0,0.0,0.0,0.583333,0.416667,3.0,0.125,9.083333,1.909169,14.0,2018-01-28 19:53:59,2017-08-11 19:09:18,191.250000,59.496620,5.246094,1.341797,0.006020,0.218628,57.0,10.0,1.0,2.375,2.550781,15.0,14.0,24.0,4.0,2.0,13.0,170.0,0.140351,0.030679,0.333333,0.090454,0.064758,0.931818,0.0,0.0,0.0,0.0,0.0,0.573864,0.426136,0.75,0.068182,9.25,0.0,3.500000,185.712121,8.864729,1.311523,0.618652,0.041412,0.203613,14.25,5.0,1.0,2.105469,1.649414,5.75,5.75,6.000000,2.000000,

In [10]:
#========================================================================
# Save Feature
#========================================================================

ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]

for df_feat, fname in zip(feat_list[1:], ['l0_1', 'l-1_0', 'lag-2_-1', 'l-3_-2', 'l-4_-3', 'l-5_-3']):
#     fname = '209_' + fname
    fname = '209'
    for col in df_feat.columns:
        if col in ignore_features: continue
        if (col.count('feature_')):continue
        if (col.count('purchase_date')) and not(col.count('diff')):continue
        feature = df_feat[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
#         utils.to_pkl_gzip(path = f'../features/5_tmp/{fname}_{col}@', obj=feature)

In [4]:
import glob
import shutil
path_list = glob.glob('../features/5_tmp/*.gz')

for path in path_list:
    tmp = utils.read_pkl_gzip(path)
    if tmp.shape[0]>330000:
        shutil.move(path, '../features/6_subset/')